In [8]:
import csv
import os
import pandas as pd

clusters = []
clusterNames = []

def get_cluster(keyword):
    for cluster in clusters:
        for key, value in cluster.items():
            if(keyword in value):
                return key
    return "None"

def write_to_csv(fileName,urlDict):
    fileName = fileName.split('/')[1]
    if (type(urlDict) is dict):
        data = pd.DataFrame([urlDict], columns=urlDict.keys())
        # if file does not exist write header 
        if not os.path.isfile('ClusterScore/' + fileName):
            data.to_csv('ClusterScore/' + fileName, header='column_names')
        else: # else it exists so append without writing the header
            data.to_csv('ClusterScore/' + fileName, mode='a', header=False)
    else:
        print("exception?")
    
def calculate_clusters_scores(rowDict):
    finTechClusters = ['cluster1_tech_words.csv']
    for key, value in rowDict.items():
        if(key in clusterNames and value > 0):
            if(key in finTechClusters):
                score = rowDict['FinTech Score']
                newScore = score + 1
                rowDict['FinTech Score'] = newScore
            else:
                score = rowDict['Non-Fintech Score']
                newScore = score + 1
                rowDict['Non-Fintech Score'] = newScore
    return rowDict
        
        

def classify_banks(filename):
    dict_list = []
    
    with open(filename, 'r', encoding = 'utf-8') as f:
        reader = csv.DictReader(f, skipinitialspace=True)
        d = {name: [] for name in reader.fieldnames}
        for row in reader:
            rowDict = {'Bank Name' : '','Job Title': '','URL (URL of the job posting)':'', 'FinTech Score' : 0, 'Non-Fintech Score':0}
            for cluster in clusterNames:
                rowDict[cluster] = 0
            for name in reader.fieldnames:
                if(name in ['Bank Name','Job Title','URL (URL of the job posting)']):
                    rowDict[name] = row[name]
                else:
                    if(int(row[name]) > 0):
                        cluster = get_cluster(name)
                        if(cluster != "None"):
                            rowDict[cluster] = 1
            write_to_csv(filename,calculate_clusters_scores(rowDict))
    
    
def classify_all():
    for filename in os.listdir("Output"):
        if filename.endswith(".csv"):
            print(filename)
            classify_banks(("Output/" + filename))
            break
    

def get_all_clusters():
    for filename in os.listdir("Clusters"):
        if filename.endswith(".csv"):
            with open("Clusters/" + filename, 'r', encoding = 'utf-8') as f:
                reader = csv.reader(f)
                keywords = list(reader)
            clusters.append({filename : [i[0] for i in keywords]})
            clusterNames.append(filename)
        
if __name__ == '__main__':
    get_all_clusters()
    classify_all()

Team10_keycorp_fifththird.csv
